In [ ]:
from dateutil.relativedelta import relativedelta
import requests
import pandas as pd
from datetime import datetime
import pytz

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Se si usa solo un sottoinsieme (chosen airports) conviene usare questo codice così non fa doppioni su dataset, perche codice fa andata ritorno
chosen_airports = ['BVA', 'BCN']

import itertools
routes = []
for subset in itertools.combinations(chosen_airports, 2):
    routes.append(subset)
print(routes)

[('BVA', 'BCN')]


In [ ]:
now = datetime.now() # current date and time

tz_Rome = pytz.timezone('Europe/Rome')
datetime_Rome = datetime.now(tz_Rome)
hour = datetime_Rome.strftime("%H:%M")
date = now.strftime("%m:%d:%Y")
date_time = date + '-' + hour

In [ ]:
# Get flights information for the airports chosen

today = datetime.today()
master_df = pd.DataFrame(columns = ['Flight Code', 'Origin','Destination','Period','Date','Departure Time', 'Arrival Time', 'Price','Published Price', 'Tracking Date', 'Tracking Hour'])
date_dict = {}

for keys, values in routes:
    for i in range(34):
        day = today + relativedelta(days=+i)
        date_dict[str(i) + 'day'] = day
    two_months = today + relativedelta(months=+2)
    date_dict['Two Months'] = two_months
    three_months = today + relativedelta(months=+3)
    date_dict['Three Months'] = three_months
    four_month = today + relativedelta(months=+4)
    date_dict['Four Months'] = four_month
    for keys2, values2  in date_dict.items():
        date_in = values2.strftime('%Y-%m-%d')
        date_out = values2 + relativedelta(days=+0)
        date_out = date_out.strftime('%Y-%m-%d')
        url = 'https://www.ryanair.com/api/booking/v4/en-gb/availability?ADT=1&CHD=0&DateIn={}&DateOut={}&Destination={}&Disc=0&INF=0&Origin={}&TEEN=0&promoCode=&IncludeConnectingFlights=false&RoundTrip=false&ToUs=AGREED'.format(date_in, date_out, values, keys)
        content = str(requests.get(url).content)
        flight_keys, prices, published_prices = ([] for i in range(3)) 
        for i, _ in enumerate(content):
            needle1 = '"flightKey":"'
            needle2 = '"amount":'
            needle3 = '"publishedFare":'
            if content[i:i + len(needle1)] == needle1:
                flight_keys.append(content[i+13:i+200].split('~~","',1)[0])
            if content[i:i + len(needle2)] == needle2:
                prices.append(content[i+9:i+18].split(',',1)[0])
            if content[i:i + len(needle3)] == needle3:
                published_prices.append(content[i+16:i+25].split(',',1)[0])
        if len(flight_keys) == len(published_prices) == len(prices):
            for i in range(0, len(flight_keys)):
                flight_code = flight_keys[i][:7]
                departure_time = flight_keys[i][-26:-21]
                arrival_time = flight_keys[i][-5:]
                date  = flight_keys[i][-16:-6]
                if flight_keys[i][-20:-17] == values:
                    origin = keys
                    destination = values
                else:
                    origin = values
                    destination = keys
                price = prices[i]
                published_price = published_prices[i]
                row = [flight_code, origin,destination,keys2,date,departure_time,arrival_time,price,published_price, date, hour]
                master_df.loc[len(master_df)] = row
print(master_df)

Empty DataFrame
Columns: [Flight Code, Origin, Destination, Period, Date, Departure Time, Arrival Time, Price, Published Price, Tracking Date, Tracking Hour]
Index: []


In [ ]:
master_df.to_csv('/content/drive/MyDrive/BVABCN/data'+date_time+'.csv')